# Accident Propensity Index Calculation v07
--- Ready for Deployment ---

Including efficiency features. Assumes that splines are not longer than around 5.5km since for each start point, only accidents within a radius of, in the most sub-optimal case, 5.5km are checked. This takes .08 seconds. When using 10km radius, it takes .17 seconds. With 100km radius it thakes .36 seconds.

Imports json now and exports json with API calculated.

## Import libraries

In [28]:
# Import relevant libraries
import pandas as pd
import math
import folium
import numpy as np
import json

## Efficiency variables

In [29]:
# Define the bucket size: 1 would be every longitude/latitude (ca. 110km); 10 is 1/10th (ca. 11km); 100 is 1/100th (ca. 1.1km); 1000 is 1/1000th (ca. 0.11km)
bucket_length = 1000

# Maximal distance of accidents from route in kilometers
max_distance = 0.05

## Split accident data into buckets - run once at start

In [30]:
# Read the dataframe with the accident locations -> has only columns ID, Start_Lat, and Start_Lng
raw_data = pd.read_csv('final_accident_data.csv')

# Functions to generate latitude and longitude IDs for the bucketing
def get_lat_id(lat):
    return int(lat*bucket_length)
def get_lng_id(lng):
    return int(lng*bucket_length)

# Assign bucketing IDs to each accident
raw_data["lat_id"] = raw_data["Start_Lat"].apply(get_lat_id)
raw_data["lng_id"] = raw_data["Start_Lng"].apply(get_lng_id)

# Group by bucketing IDs
groups = raw_data.groupby(['lat_id', 'lng_id'])

# Iterate over the groups and create individual dataframes
for name, group in groups:
    # Create the dataframe name
    df_name = f"accidents_{name[0]}_{name[1]}"
    
    # Create the dataframe
    vars()[df_name] = group.copy()

# Get a copy of all global variables
global_vars = globals().copy()

## Functions to calculate distances and find accidents on route - run once at start

In [31]:
# Define a function to calculate the distance between two points
def distance(point1, point2):
    lat1, lon1 = point1
    lat2, lon2 = point2
    km_per_lat = 110.574 # km per degree latitude
    km_per_lon = 111.320 # km per degree longitude at the equator
    dx = (lon2 - lon1) * km_per_lon * math.cos((lat1 + lat2) / 2)
    dy = (lat2 - lat1) * km_per_lat
    return math.sqrt(dx**2 + dy**2)

# Define a function to calculate the distance between a point and a line segment
def distance_to_segment(point, segment_start, segment_end):
    px, py = point
    x1, y1 = segment_start
    x2, y2 = segment_end
    dx, dy = x2 - x1, y2 - y1
    segment_length_squared = dx*dx + dy*dy
    if segment_length_squared == 0:
        return distance(point, segment_start)
    t = max(0, min(1, ((px - x1) * dx + (py - y1) * dy) / segment_length_squared))
    x = x1 + t * dx
    y = y1 + t * dy
    return distance(point, (x, y))

# Generate dataframe of accidents close to the segment
def generate_data(start_point):
    # Extract the integer values of the start point lat and lng
    start_lat = int(start_point[0]*bucket_length)
    start_lng = int(start_point[1]*bucket_length)

    # Get the dataframes that match the criteria
    dfs_to_use = []
    for lat_offset in [-1, 0, 1]:
        for lng_offset in [-1, 0, 1]:
            lat_id = start_lat + lat_offset
            lng_id = start_lng + lng_offset
            df_name = f"accidents_{lat_id}_{lng_id}"
            if df_name in global_vars and isinstance(global_vars[df_name], pd.DataFrame):
                dfs_to_use.append(global_vars[df_name])

    # Concatenate the dataframes and reset the index
    combined_df = pd.concat(dfs_to_use)
    data = combined_df.reset_index(drop=True)
    return data

# Define a function to find accidents on a given route within a maximum distance
def find_accidents_on_route(start_point, end_point):
    # Create a mask for accidents that are within the maximum distance from the route
    data = generate_data(start_point)
    mask = data.apply(lambda row: distance_to_segment((row['Start_Lat'], row['Start_Lng']), start_point, end_point) <= max_distance, axis=1)

    # Return the accidents that match the mask
    accidents = data.loc[mask]
    return accidents

## Find accidents on route - run every time

In [32]:
# load data from json file
with open('route_data.json') as f:
    data = json.load(f)

# create DataFrame from loaded data
route_data = pd.DataFrame(data)

In [39]:
# Split the DataFrame into three equally sized parts
df_list = np.array_split(route_df, 5)

# Loop through the DataFrames
for i, df in enumerate(df_list):
    # Safe each split dataframe as a new route_df_{i} dataframe
    globals()[f"route_df_{i+1}"] = df
    # Create a new DataFrame to store the results
    accidents_df = pd.DataFrame()
    # Loop through the pairs of subsequent coordinates
    for j in range(len(df) - 1):
        start_point = j
        end_point = j + 1
        # Call the function and append the results to the accidents DataFrame
        accidents_df = accidents_df.append(find_accidents_on_route(df.iloc[start_point], df.iloc[end_point]), ignore_index=True)
    # Drop duplicate rows from the accidents DataFrame
    accidents_df.drop_duplicates(inplace=True)
    
    # Assign a name to the accidents DataFrame based on the index of the original DataFrame
    df_index = i+1
    globals()[f'accidents_df_{df_index}'] = accidents_df
    
    # Print the resulting accidents DataFrame
    # print(f'accidents_df_{df_index}:')
    # print(accidents_df)

/var/folders/2z/5xl96y_10clcjhbp86qg3fyr0000gn/T/ipykernel_2387/2438444131.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  accidents_df = accidents_df.append(find_accidents_on_route(df.iloc[start_point], df.iloc[end_point]), ignore_index=True)
/var/folders/2z/5xl96y_10clcjhbp86qg3fyr0000gn/T/ipykernel_2387/2438444131.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  accidents_df = accidents_df.append(find_accidents_on_route(df.iloc[start_point], df.iloc[end_point]), ignore_index=True)
/var/folders/2z/5xl96y_10clcjhbp86qg3fyr0000gn/T/ipykernel_2387/2438444131.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  accidents_df = accidents_df.append(find_accidents_on_route(df.iloc[start_point], df.iloc[end_point]), ignor

## Calculate Accident Propensity Index

In [40]:
# create a list of dataframes
accidents_dfs = [accidents_df_1, accidents_df_2, accidents_df_3, accidents_df_4, accidents_df_5]
api_dict = {}

# loop through the list of dataframes and join each one with the all_accidents dataframe
for i, df in enumerate(accidents_dfs, start=1):
    exec(f'total_severity = accidents_df_{i}["Severity"].sum()')
    api = total_severity / 6035011
    api_dict[f'api_{i}'] = round(api,8)

In [44]:
route_dfs = [route_df_1, route_df_2, route_df_3, route_df_4, route_df_5]

# Create a dictionary to store the JSON data
json_dict = {}

# Loop over the segments and add the API and route data to the JSON dictionary
for i in range(5):
    segment_name = f"segment_{i+1}"
    json_dict[segment_name] = {}
    json_dict[segment_name]["api"] = api_dict[f"api_{i+1}"]
    json_dict[segment_name]["route"] = route_dfs[i].to_dict(orient="records")

# Write the JSON data to a file
with open("backend_output.json", "w") as f:
    json.dump(json_dict, f)